To-Do
- [ ] Create Separate Functions for each task
- [ ] Evaluation of our model
- [ ] How to generalize the model?
- [ ] More about preprocessing
- [ ] Improve Handling of rare words

In [ ]:
!pip install keras-preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install -U tensorflow-text==2.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pathlib
from keras.utils import to_categorical
import pickle
import tensorflow as tf
import tensorflow_text as tf_text
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Embedding, SimpleRNN, RepeatVector, TimeDistributed

In [ ]:
tf.__version__

'2.11.0'

## English to Spanish

In [ ]:
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

2638744/2638744 [==============================] - 0s 0us/step


In [ ]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  context = np.array([context for target, context in pairs])
  target = np.array([target for target, context in pairs])

  return target, context

data = load_data(path_to_file)

In [ ]:
inputs = np.array(data[0][:10000])
outputs = np.array(data[1][:10000])

In [ ]:
# vectorizer = tf.keras.layers.TextVectorization(standardize="lower_and_strip_punctuation", output_sequence_length=src)
# text_dataset = tf.data.Dataset.from_tensor_slices(inputs)
# vectorizer.adapt(text_dataset)

In [ ]:
# # dump(vectorizer,open('drive/MyDrive/Machine Translation/vectroizer.pkl','wb'))
# import pickle
# pickle.dump({'config': vectorizer.get_config(),
#              'weights': vectorizer.get_weights()}
#             , open("drive/MyDrive/Machine Translation/vectorizer.pkl", "wb"))

In [ ]:

# saved = pickle.load(open('drive/MyDrive/Machine Translation/vectorizer.pkl','rb'))
# vectorizer = tf.keras.layers.TextVectorization.from_config(saved['config'])

# vectorizer.adapt(tf.data.Dataset.from_tensor_slices(['random']))
# vectorizer.set_weights(saved['weights'])

In [ ]:
# spanish_vectorizer = tf.keras.layers.TextVectorization(standardize="lower_and_strip_punctuation", output_sequence_length=tar)
# text_dataset = tf.data.Dataset.from_tensor_slices(outputs)
# spanish_vectorizer.adapt(text_dataset)

In [ ]:
# import pickle
# pickle.dump({'config': spanish_vectorizer.get_config(),
#              'weights': spanish_vectorizer.get_weights()}
#             , open("drive/MyDrive/Machine Translation/spanish_vectorizer.pkl", "wb"))

In [ ]:
# saved = pickle.load(open('drive/MyDrive/Machine Translation/spanish_vectorizer.pkl','rb'))
# spanish_vectorizer = tf.keras.layers.TextVectorization.from_config(saved['config'])

# spanish_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(['random']))
# spanish_vectorizer.set_weights(saved['weights'])

In [ ]:
# train_X = vectorizer(inputs[indices])
# train_Y = spanish_vectorizer(outputs[indices])
# # model.fit(train_X, train_Y)

In [ ]:
def create_tokenizer(source, target, num_words=5000, lower=True):
  src_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words,lower=lower)
  tar_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words,lower=lower)

  src_tokenizer.fit_on_texts(source)
  tar_tokenizer.fit_on_texts(target)
  return src_tokenizer, tar_tokenizer

In [ ]:
eng_tokenizer,spa_tokenizer = create_tokenizer(inputs, outputs)

In [ ]:
def transform_data(source, target):
  source = eng_tokenizer.texts_to_sequences(inputs)
  source = tf.keras.preprocessing.sequence.pad_sequences(source, maxlen=8, padding='post')

  target = spa_tokenizer.texts_to_sequences(outputs)
  target = tf.keras.preprocessing.sequence.pad_sequences(target, maxlen=8, padding='post')
  return source, target

def split_data(source, target, train_split=0.8):
  size = inputs.shape[0]
  indices = np.random.randint(0, size, size=(int(size*train_split),))

  trainX = source[indices]
  trainY = target[indices]

  mask = np.ones(size,bool)
  mask[indices] = False
  testX = source[mask]
  testY = target[mask]
  return trainX, trainY, testX, testY

source, target = transform_data(inputs, outputs)
trainX, trainY, testX, testY = split_data(source, target, 0.8)

In [ ]:
def encode_output(sequences, vocab_size):
 ylist = list()
 for sequence in sequences:
  encoded = tf.keras.utils.to_categorical(sequence, num_classes=vocab_size)
  ylist.append(encoded)
 y = np.array(ylist)
 y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
 return y

trainY = encode_output(trainY, len(spa_tokenizer.word_index)+1)

In [ ]:
tar_vocab = len(spa_tokenizer.word_index)+1
src_vocab = len(eng_tokenizer.word_index)+1
src = max(len(line.split()) for line in inputs)
tar = max(len(line.split()) for line in outputs)

In [ ]:
layer1 = Embedding(1000, 16, input_length=8)
layer2 = LSTM(256)
model = Sequential()
model.add(layer1)
model.add(layer2)

# we want to replicate the context vector for each time step
model.add(RepeatVector(8))
model.add(LSTM(256, return_sequences=True))

# converting decoder output to our desired sequence format
model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 16)             16000     
                                                                 
 lstm (LSTM)                 (None, 256)               279552    
                                                                 
 repeat_vector (RepeatVector  (None, 8, 256)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 8, 256)            525312    
                                                                 
 time_distributed (TimeDistr  (None, 8, 4961)          1274977   
 ibuted)                                                         
                                                                 
Total params: 2,095,841
Trainable params: 2,095,841
Non-

In [ ]:
model.fit(trainX, trainY, epochs=30, batch_size=64)

Epoch 1/30
125/125 [==============================] - 14s 45ms/step - loss: 3.3985
Epoch 2/30
125/125 [==============================] - 2s 16ms/step - loss: 2.5767
Epoch 3/30
125/125 [==============================] - 2s 14ms/step - loss: 2.5141
Epoch 4/30
125/125 [==============================] - 2s 13ms/step - loss: 2.4538
Epoch 5/30
125/125 [==============================] - 2s 13ms/step - loss: 2.3953
Epoch 6/30
125/125 [==============================] - 2s 12ms/step - loss: 2.3504
Epoch 7/30
125/125 [==============================] - 2s 14ms/step - loss: 2.3107
Epoch 8/30
125/125 [==============================] - 2s 15ms/step - loss: 2.2712
Epoch 9/30
125/125 [==============================] - 2s 12ms/step - loss: 2.2328
Epoch 10/30
125/125 [==============================] - 1s 12ms/step - loss: 2.2040
Epoch 11/30
125/125 [==============================] - 2s 13ms/step - loss: 2.1764
Epoch 12/30
125/125 [==============================] - 1s 12ms/step - loss: 2.1483
Epoch 13/30


In [ ]:
def inference(model, input, source_tokenizer, tar_token_to_word):
  """
  Args:
  input - a string in the source language
  """
  # tokenized = source_tokenizer.texts_to_sequences(input)
  # test = tf.keras.preprocessing.sequence.pad_sequences(tokenized, maxlen=8, padding='post')
  prediction = model.predict(input)

  output = [np.argmax(vector) for vector in prediction[0]]

  output_list = []
  for i in output:
    if i == 0:
      break
    else:
      output_list.append(tar_token_to_word[i])

  output_sentence = ' '.join(output_list)
  return output_sentence

eng_word_to_token = eng_tokenizer.word_index
eng_token_to_word = {token:word for word, token in eng_word_to_token.items()}

spa_word_to_token = spa_tokenizer.word_index
spa_token_to_word = {token:word for word, token in spa_word_to_token.items()}

In [ ]:
inference(model, trainX[1:3], eng_tokenizer, spa_token_to_word)

1/1 [==============================] - 1s 640ms/step


'me llamé'

## Eng to Hin (Incomplete)

In [ ]:
with open('hin.txt') as f:
    lines = f.readlines()

In [ ]:
len(lines)

2909

In [ ]:
def prepare_data(lines):
  inputs = []
  outputs = []

  for i in range(len(lines)):
    src, target, _ = lines[i].split("\t")
    inputs.append(src)
    outputs.append(target)
  
  inputs = np.array(inputs)
  outputs = np.array(outputs)
  return inputs, outputs

inputs, outputs = prepare_data(lines)

In [ ]:
inputs.shape, outputs.shape

((2909,), (2909,))

In [ ]:
eng_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000,lower=True)
hin_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000,lower=True)

eng_tokenizer.fit_on_texts(inputs)
hin_tokenizer.fit_on_texts(outputs)

trainX = eng_tokenizer.texts_to_sequences(inputs)
trainX = tf.keras.preprocessing.sequence.pad_sequences(trainX, maxlen=8, padding='post')

trainY = hin_tokenizer.texts_to_sequences(outputs)
trainY = tf.keras.preprocessing.sequence.pad_sequences(trainY, maxlen=8, padding='post')

def encode_output(sequences, vocab_size):
 ylist = list()
 for sequence in sequences:
  encoded = tf.keras.utils.to_categorical(sequence, num_classes=vocab_size)
  ylist.append(encoded)
 y = np.array(ylist)
 y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
 return y

trainY = encode_output(trainY[:9000], len(hin_tokenizer.word_index)+1)

In [ ]:
len(hin_tokenizer.word_index)

3012

In [ ]:
tar_vocab = len(hin_tokenizer.word_index)+1
src_vocab = len(eng_tokenizer.word_index)+1
src = max(len(line.split()) for line in inputs)
tar = max(len(line.split()) for line in outputs)

In [ ]:
layer1 = Embedding(src_vocab, 16, input_length=8)
layer2 = LSTM(256)
model = Sequential()
model.add(layer1)
model.add(layer2)

# we want to replicate the context vector for each time step
model.add(RepeatVector(8))
model.add(LSTM(256, return_sequences=True))

# converting decoder output to our desired sequence format
model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 16)             38336     
                                                                 
 lstm (LSTM)                 (None, 256)               279552    
                                                                 
 repeat_vector (RepeatVector  (None, 8, 256)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 8, 256)            525312    
                                                                 
 time_distributed (TimeDistr  (None, 8, 3013)          774341    
 ibuted)                                                         
                                                                 
Total params: 1,617,541
Trainable params: 1,617,541
Non-

In [ ]:
trainX.shape

(2909, 8)

In [ ]:
!pip install tensorflow-addons==0.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00


In [ ]:
import tensorflow_addons as tfa

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
bi_model = Sequential()
bi_model.add(Embedding(src_vocab, 16, input_length=8))
bi_model.add(tf.keras.layers.Bidirectional(LSTM(256, return_sequences=True), input_shape=(8,16)))
bi_model.add(tf.keras.layers.Attention(256))
bi_model.add(LSTM(256, return_sequences=True))
bi_model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
bi_model.compile(optimizer='adam', loss='categorical_crossentropy')
bi_model.summary()

ValueError: ignored

In [ ]:
trainY.shape

(2909, 8, 3013)

In [ ]:
bi_model.fit(trainX, trainY, epochs=250, batch_size=64)

Epoch 1/250
46/46 [==============================] - 1s 13ms/step - loss: 0.8828
Epoch 2/250
46/46 [==============================] - 1s 11ms/step - loss: 0.8536
Epoch 3/250
46/46 [==============================] - 0s 11ms/step - loss: 0.8345
Epoch 4/250
46/46 [==============================] - 1s 14ms/step - loss: 0.8100
Epoch 5/250
46/46 [==============================] - 1s 11ms/step - loss: 0.8015
Epoch 6/250
46/46 [==============================] - 1s 14ms/step - loss: 0.7893
Epoch 7/250
46/46 [==============================] - 1s 15ms/step - loss: 0.7810
Epoch 8/250
46/46 [==============================] - 1s 20ms/step - loss: 0.7724
Epoch 9/250
46/46 [==============================] - 1s 12ms/step - loss: 0.7632
Epoch 10/250
46/46 [==============================] - 0s 11ms/step - loss: 0.7563
Epoch 11/250
46/46 [==============================] - 0s 11ms/step - loss: 0.7511
Epoch 12/250
46/46 [==============================] - 1s 11ms/step - loss: 0.7441
Epoch 13/250
46/46 [=====

In [ ]:
predictions = bi_model.predict(trainX)

91/91 [==============================] - 3s 3ms/step


In [ ]:
predictions.shape

(2909, 8, 3013)

In [ ]:
[np.argmax(vector) for vector in predictions[0]]

[775, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
model.fit(trainX, trainY, epochs=250, batch_size=64)

Epoch 1/250
46/46 [==============================] - 16s 124ms/step - loss: 6.3958
Epoch 2/250
46/46 [==============================] - 2s 38ms/step - loss: 5.4829
Epoch 3/250
46/46 [==============================] - 1s 22ms/step - loss: 5.2873
Epoch 4/250
46/46 [==============================] - 1s 20ms/step - loss: 5.1674
Epoch 5/250
46/46 [==============================] - 1s 16ms/step - loss: 5.0356
Epoch 6/250
46/46 [==============================] - 1s 13ms/step - loss: 4.9334
Epoch 7/250
46/46 [==============================] - 1s 20ms/step - loss: 4.8706
Epoch 8/250
46/46 [==============================] - 1s 17ms/step - loss: 4.8168
Epoch 9/250
46/46 [==============================] - 1s 23ms/step - loss: 4.7599
Epoch 10/250
46/46 [==============================] - 1s 13ms/step - loss: 4.7259
Epoch 11/250
46/46 [==============================] - 0s 9ms/step - loss: 4.6908
Epoch 12/250
46/46 [==============================] - 1s 17ms/step - loss: 4.6545
Epoch 13/250
46/46 [====

In [ ]:
input = trainX[121:528]

In [ ]:
# mapping tokens to words and vice-versa for both source and the target
eng_word_to_token = eng_tokenizer.word_index
eng_token_to_word = {token:word for word, token in eng_word_to_token.items()}

hin_word_to_token = hin_tokenizer.word_index
hin_token_to_word = {token:word for word, token in hin_word_to_token.items()}

In [ ]:
def inference(model, input, source_tokenizer, tar_token_to_word):
  """
  Args:
  input - a string in the source language
  """
  tokenized = source_tokenizer.texts_to_sequences(test)
  test = tf.keras.preprocessing.sequence.pad_sequences(tokenized, maxlen=8, padding='post')
  prediction = model.predict(test)

  output = [np.argmax(vector) for vector in prediction[0]]

  output_list = []
  for i in output:
    if i == 0:
      break
    else:
      output_list.append(tar_token_to_word[i])

  output_sentence = ' '.join(output_list)
  return output_sentence